In [1]:
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from berny import Berny, geomlib, coords,Geometry,Math
from FcMole import FcM,FcM_like
from AP_class import APDFT_perturbator
from alch_deriv import alch_deriv
from pyscf import gto,scf
import numpy as np 
from numpy import dot,log
from numpy.linalg import norm as norm
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
from pyscf.geomopt.berny_solver import to_berny_geom as tbg
import basis_set_exchange as bse
from FDcoeffs import *
from pyscf.geomopt.berny_solver import optimize
ang2bohr=1.8897261246
bohr2ang=.5291772109
np.set_printoptions(precision=3)

In [2]:
import ipyvolume as ipv
from mpl_toolkits.mplot3d import Axes3D

In [3]:
import berny.coords
import sys
sys.path.append("/home/administrator/Documenti/APDFT/prototyping/hessian/AAFF/Libs/")
from Morse import mpa
from AP_class import APDFT_perturbator as APP

In [4]:
def anglecenter(coords):
    return np.around(np.arccos(coords[1].dot(coords[2])/norm(coords[1])/norm(coords[2])),decimals=3)
def cnorm(c):
    return norm(c[1]-c[0])

In [5]:
def build_h_ic(s,g_ic,h0,B_inv):
    geom0=s.geom.copy()
    B=s.coords.B_matrix(geom0)
    bms=[]
    for i in range(geom0.coords.flatten().shape[0]):
        a=geom0.coords.flatten()
        geom1=geom0.copy()
        a[i]+=.001*bohr2ang
        a=a.reshape(geom0.coords.shape)
        geom1.coords=a
        bms.append((s.coords.B_matrix(geom1)-B)*1000)
    bms_arr=np.asarray(bms)
    BpG2=np.einsum('ijk,j->ik',bms,g_ic)
    h_ic=B_inv.T@(h0-BpG2)@B_inv
    return h_ic

In [69]:
tbbde={"CH":105/627.5,"HC":105/627.5,"NH":110/627.5,"HN":110/627.5,\
       "OH":119/627.5,"HO":119/627.5,"FH":136/627.5,"HF":136/627.5}
def mpa_pb(coords,atoml,g,h,gic=False,solve_ic=False):
    g=np.asarray(g)
    if not len(h.shape)==2:
        h=h.swapaxes(1,2)
        h=h.reshape(g.shape[0]*3,g.shape[0]*3)
    geom0=Geometry(atoml,coords*bohr2ang)
    bernyobj=Berny(geom0)
    s=bernyobj._state
    B = s.coords.B_matrix(geom0)
    q0=s.coords.eval_geom(geom0)
    B_inv = B.T.dot(Math.pinv(np.dot(B, B.T)))
    if not gic:
        g_ic=np.dot(B_inv.T, (g).reshape(-1))
    else:
        g_ic=g
   # print(g_ic)
    h_ic=build_h_ic(s,g_ic,h,B_inv)
  #  print (s.coords._coords)
    ddq_mb=np.zeros_like(q0)
    for i in range(len(s.coords._coords)):
        if s.coords._coords[i].__class__ is berny.coords.Bond:
            bondatoms=atoml[s.coords._coords[i].i]+atoml[s.coords._coords[i].j]
            ddq_mb[i]+=mpa(g_ic[i],h_ic[i,i],q0[i],tbbde[bondatoms])[0]+g_ic[i]/h_ic[i,i]-q0[i]
    if not solve_ic:    return ddq_mb[0]
    else: 
        dq_NR=-np.linalg.solve(h_ic,g_ic)
        return q0,q0+dq_NR,q0+dq_NR+ddq_mb
    #ddq_mb=np.asarray([mpa(g_ic[0],h_ic[0,0],q0[0],tbde)[0]+g_ic[0]/h_ic[0,0]

In [7]:
l=3.3376
x=-(1/24)**.5*l
z=(2/3)**.5*l+x
ch4str="C 0 0 0; H 0 0  {}; H  0  {} {}; H {} {} {}; H {} {} {}".format(z,l/3**.5,x,l/2,-l*3**.5/6,x,-l/2,-l*3**.5/6,x)

In [8]:
CH4=gto.M(atom=ch4str,unit="bohr",basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
mf = scf.RHF(CH4)
mf.scf()

converged SCF energy = -40.2151804034984


-40.21518040349835

In [9]:
g0=mf.Gradients().grad()

--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000     0.0000000000    -0.0000000000
1 H     0.0000000000     0.0000000000    -0.0000007926
2 H     0.0000000000    -0.0000007473     0.0000002642
3 H    -0.0000006472     0.0000003736     0.0000002642
4 H     0.0000006472     0.0000003736     0.0000002642
----------------------------------------------


In [10]:
h0=mf.Hessian().hess()

In [11]:
ch4coords=CH4.atom_coords()

In [12]:
afC=aaff(mf,[1,0,0,0,0])+alc_deriv_grad_nuc(CH4,[1,0,0,0,0])
afH1=aaff(mf,[0,0,0,0,-1])+alc_deriv_grad_nuc(CH4,[0,0,0,0,-1])
afH2=aaff(mf,[0,0,0,-1,0])+alc_deriv_grad_nuc(CH4,[0,0,0,-1,0])
afH3=aaff(mf,[0,0,-1,0,0])+alc_deriv_grad_nuc(CH4,[0,0,-1,0,0])

In [28]:
nh3optg,oh2optg,fhoptg=np.load("optgeoms.npy")

array([[0.   , 0.   , 0.   ],
       [0.   , 0.   , 1.696]])

In [30]:
#getting inverse of the hessian matrix
H1=np.linalg.inv((np.swapaxes(h0,1,2)).reshape(15,15))
h1=H1.reshape(5,3,5,3)

In [31]:
nh3predictg=(ch4coords-np.einsum('ijkl,kl->ij',h1,  afC+afH1))[:4]
oh2predictg=(ch4coords-np.einsum('ijkl,kl->ij',h1,  2*afC+afH1+afH2))[:3]
fhpredictg=(ch4coords-np.einsum('ijkl,kl->ij',h1,  3*afC+afH1+afH2+afH3))[:2]

In [32]:
#place non H atom in the origin
nh3predictg-=nh3predictg[0]
oh2predictg-=oh2predictg[0]
fhpredictg-=fhpredictg[0]

In [71]:
#bond lengths
print(norm(ch4coords[1]))
print(norm(nh3predictg[1]),norm(nh3predictg[1])+mpa_pb(ch4coords,["N","H","H","H","H"],(afC+afH1),(np.swapaxes(h0,1,2)).reshape(15,15))\
      ,norm(nh3optg[1]))
print(norm(oh2predictg[1]),norm(oh2predictg[1])+mpa_pb(ch4coords,["O","H","H","H","H"],(2*afC+afH1+afH2),(np.swapaxes(h0,1,2)).reshape(15,15))\
      ,norm(oh2optg[1]))
print(norm(fhpredictg[1]),norm(fhpredictg[1])+mpa_pb(ch4coords,["F","H","H","H","H"],(3*afC+afH1+afH2+afH3),(np.swapaxes(h0,1,2)).reshape(15,15))\
      ,norm(fhoptg[1]))

2.043854241378284
1.877102610434938 1.9276049907831783 1.8856061076490165
1.7017425238100001 1.8819527137059242 1.7756053665813991
1.5148043094330303 1.8608835940133732 1.69574139


In [72]:
mpa_pb(ch4coords,["N","H","H","H","H"],(afC+afH1),(np.swapaxes(h0,1,2)).reshape(15,15),solve_ic=True)

(array([2.044, 2.044, 2.044, 2.044, 1.911, 1.911, 1.911, 1.911, 1.911,
        1.911]),
 array([1.875, 1.875, 1.875, 2.038, 2.135, 2.135, 2.411, 2.135, 2.411,
        2.411]),
 array([1.926, 1.926, 1.926, 2.038, 2.135, 2.135, 2.411, 2.135, 2.411,
        2.411]))

In [48]:
mfp1=scf.RHF(FcM_like(CH4,fcs=[1,0,0,0,-1]))
mfp1.scf(dm0=mfp1.init_guess_by_1e())
gp1=mfp1.Gradients().grad()
hp1=mfp1.Hessian().hess()

s
converged SCF energy = -56.2053208898248
--------------- RHF gradients ---------------
         x                y                z
0 C    -0.0379252172    -0.0218961344    -0.0154829051
1 H    -0.0052084464    -0.0030070979     0.0634324768
2 H    -0.0052084464     0.0588023467    -0.0239792781
3 H     0.0483201028    -0.0339118203    -0.0239792781
4 H     0.0000220072     0.0000127059     0.0000089844
----------------------------------------------


In [49]:
mfp2=scf.RHF(FcM_like(CH4,fcs=[2,0,0,-1,-1]))
mfp2.scf(dm0=mfp2.init_guess_by_1e())
gp2=mfp2.Gradients().grad()
hp2=mfp2.Hessian().hess()

s
converged SCF energy = -76.0272830766302
--------------- RHF gradients ---------------
         x                y                z
0 C    -0.0000000000    -0.0866717430    -0.0612861772
1 H     0.0000000000    -0.0101452050     0.1063794599
2 H     0.0000000000     0.0969137816    -0.0450248110
3 H     0.0000567099    -0.0000484168    -0.0000342358
4 H    -0.0000567099    -0.0000484168    -0.0000342358
----------------------------------------------


In [50]:
mfp3=scf.RHF(FcM_like(CH4,fcs=[3,0,-1,-1,-1]))
mfp3.scf(dm0=mfp3.init_guess_by_1e())
gp3=mfp3.Gradients().grad()
hp3=mfp3.Hessian().hess()

s
converged SCF energy = -100.027837815927
--------------- RHF gradients ---------------
         x                y                z
0 C    -0.0000000351    -0.0000002003    -0.1356407882
1 H     0.0000000354     0.0000002023     0.1358085065
2 H    -0.0000000002     0.0002073815    -0.0000559045
3 H     0.0001795974    -0.0001036918    -0.0000559066
4 H    -0.0001795975    -0.0001036916    -0.0000559071
----------------------------------------------


In [51]:
p1cds=ch4coords-np.linalg.solve((np.swapaxes(hp1,1,2)).reshape(15,15),gp1.reshape(-1)).reshape(5,3)
p1cds-=p1cds[0]
p2cds=ch4coords-np.linalg.solve((np.swapaxes(hp2,1,2)).reshape(15,15),gp2.reshape(-1)).reshape(5,3)
p2cds-=p2cds[0]
p3cds=ch4coords-np.linalg.solve((np.swapaxes(hp3,1,2)).reshape(15,15),gp3.reshape(-1)).reshape(5,3)
p3cds-=p3cds[0]

In [70]:
norm(p1cds[1])+mpa_pb(ch4coords,["N","H","H","H","H"],gp1,(np.swapaxes(hp1,1,2)).reshape(15,15)),\
norm(p2cds[1])+mpa_pb(ch4coords,["O","H","H","H","H"],gp2,(np.swapaxes(hp2,1,2)).reshape(15,15)),\
norm(p3cds[1])+mpa_pb(ch4coords,["F","H","H","H","H"],gp3,(np.swapaxes(hp3,1,2)).reshape(15,15))

(1.8909383423668487, 1.7953046892110192, 1.7377378345683856)

In [53]:
mpa_pb(ch4coords[:4],["N","H","H","H"],gp1[:4],(np.swapaxes(hp1,1,2)).reshape(15,15)[:12,:12],solve_ic=True)

[0.063 0.063 0.063 0.012 0.012 0.012]
[Bond(0, 1, weak=0), Bond(0, 2, weak=0), Bond(0, 3, weak=0), Angle(1, 0, 2, weak=0), Angle(1, 0, 3, weak=0), Angle(2, 0, 3, weak=0)]


(array([2.044, 2.044, 2.044, 1.911, 1.911, 1.911]),
 array([1.83 , 1.83 , 1.83 , 1.911, 1.911, 1.911]),
 array([1.89 , 1.89 , 1.89 , 1.911, 1.911, 1.911]))

In [54]:
print(norm(nh3optg[1]),norm(oh2optg[1]),norm(fhoptg[1]))

1.8856061076490165 1.7756053665813991 1.69574139


In [55]:
anglecenter(p1cds),anglecenter(p2cds)

(1.905, 1.861)

In [56]:
anglecenter(ch4coords),anglecenter(nh3optg),anglecenter(oh2optg)

(1.911, 1.888, 1.857)

In [57]:
anglecenter(p1cds)*180/np.pi,anglecenter(p2cds)*180/np.pi

(109.14845997242182, 106.62744567384621)

In [58]:
anglecenter(ch4coords)*180/np.pi,anglecenter(nh3optg)*180/np.pi,anglecenter(oh2optg)*180/np.pi

(109.49223464950032, 108.17443172069942, 106.39826255579388)